In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import psycopg2
from bs4 import BeautifulSoup as bp
import time
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from shapely.geometry import Point


In [3]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

In [4]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'vi-vn',
}

In [5]:
options=Options()
options.add_argument(f"user-agent={USER_AGENT}")
driver=webdriver.Chrome(options=options)

In [6]:
driver.get('https://www.google.com/maps')

In [12]:
def get_location(address):
    # Click the search box input element
    print(f'get location for {address}')
    find = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
    find.click()
    find.clear()
    time.sleep(2)

# Input the address
    find.send_keys(address)
    time.sleep(2)
    button=driver.find_element(By.XPATH,'//*[@id="searchbox-searchbutton"]')
    button.click()
    time.sleep(2)
    current_url=driver.current_url
    current_url=current_url.split('/')
    current_url
    raw_location=current_url[6].split(',')
    raw_location
    latitude_float=float(raw_location[0].replace('@',''))
    longitude_float=float(raw_location[1])
# In ra kết quả hoặc lưu vào biến
    print(f"Latitude: {latitude_float}, Longitude: {longitude_float}")
    return {'latitude': latitude_float, 'longitude': longitude_float}

In [8]:
def insert_location(id,location):
    print(f"insert location {id} into db")
    if isinstance(location, dict) and 'longitude' in location and 'latitude' in location:
        point = Point(location['longitude'], location['latitude'])
        point_wkt = point.wkt
        conn = psycopg2.connect(postgres_url)
        cur = conn.cursor()
        cur.execute("""SET search_path TO travel_database, public;""")
        query="""
        UPDATE Hotel SET location = ST_GeomFromText(%s, 4326)
        WHERE hotel_id = %s;"""
        cur.execute(query, (point_wkt,id))
        conn.commit()
        cur.close()
        conn.close()
        print(f"updated location for {id}")
    else:
        print("error: location is not dict")

In [9]:
def get_address():
    print(f"taking address of NULL places!!!")
    conn=psycopg2.connect(postgres_url)
    cur=conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    query="""
    SELECT address,hotel_id from hotel WHERE location is NULL"""
    cur.execute(query,)
    conn.commit()
    a= cur.fetchall()
    cur.close()
    conn.close()
    print(f"found {len(a)} places")
    return a

In [13]:
if __name__=='__main__':
    a=get_address()
    for row in a:
        address=row[0]
        id=row[1]
        location=get_location(address)
        time.sleep(1)
        insert_location(id,location)
        time.sleep(2)


taking address of NULL places!!!
found 1081 places
get location for ("3 Hẻm 148 Trần Duy Hưng, Phường Trung Hòa","Cầu Giấy","Hà Nội")
Latitude: 21.0130551, Longitude: 105.7963312
insert location 65 into db
updated location for 65
get location for ("M10-L14, Khu A, KĐT Mới Dương Nội, La Khê","Hà Đông","Hà Nội")


KeyboardInterrupt: 